In [1]:
import pandas as pd

In [2]:
dataset=pd.read_csv("Social_Network_Ads.csv")

In [3]:
dataset

,User ID,Gender,Age,EstimatedSalary,Purchased
0,15624510,Male,19,19000,0
1,15810944,Male,35,20000,0
2,15668575,Female,26,43000,0
3,15603246,Female,27,57000,0
4,15804002,Male,19,76000,0
...,...,...,...,...,...
395,15691863,Female,46,41000,1
396,15706071,Male,51,23000,1
397,15654296,Female,50,20000,1
398,15755018,Male,36,33000,0


In [4]:
dataset=pd.get_dummies(dataset,dtype=int,drop_first=True)
dataset

,User ID,Age,EstimatedSalary,Purchased,Gender_Male
0,15624510,19,19000,0,1
1,15810944,35,20000,0,1
2,15668575,26,43000,0,0
3,15603246,27,57000,0,0
4,15804002,19,76000,0,1
...,...,...,...,...,...
395,15691863,46,41000,1,0
396,15706071,51,23000,1,1
397,15654296,50,20000,1,0
398,15755018,36,33000,0,1


In [5]:
dataset["Purchased"].value_counts()

Purchased
0    257
1    143
Name: count, dtype: int64

In [6]:
Independent=dataset[['User ID','Gender_Male','Age','EstimatedSalary']]
Independent

,User ID,Gender_Male,Age,EstimatedSalary
0,15624510,1,19,19000
1,15810944,1,35,20000
2,15668575,0,26,43000
3,15603246,0,27,57000
4,15804002,1,19,76000
...,...,...,...,...
395,15691863,0,46,41000
396,15706071,1,51,23000
397,15654296,0,50,20000
398,15755018,1,36,33000


In [7]:
Dependent=dataset[['Purchased']]
Dependent

,Purchased
0,0
1,0
2,0
3,0
4,0
...,...
395,1
396,1
397,1
398,0


In [8]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(Independent,Dependent,test_size=0.30,random_state=0)

In [9]:
from sklearn.preprocessing import StandardScaler
sc=StandardScaler()
x_train=sc.fit_transform(x_train)
x_test=sc.transform(x_test)

In [10]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
param_grid={'penalty':['l1','l2','elasticnet', None],'solver':['lbfgs','liblinear','saga','newton-cg']}
grid=GridSearchCV(LogisticRegression(),param_grid,refit=True,verbose=3,n_jobs=-1,scoring='f1_weighted')
grid.fit(x_train,y_train)

Fitting 5 folds for each of 16 candidates, totalling 80 fits


C:\Users\nandh\anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py:547: FitFailedWarning: 
35 fits failed out of a total of 80.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
5 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\nandh\anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py", line 895, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\nandh\anaconda3\Lib\site-packages\sklearn\base.py", line 1474, in wrapper
    return fit_method(estimator, *args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\nandh\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py", line 1172, in fit
    solver = _c

GridSearchCV(estimator=LogisticRegression(), n_jobs=-1,
             param_grid={'penalty': ['l1', 'l2', 'elasticnet', None],
                         'solver': ['lbfgs', 'liblinear', 'saga', 'newton-cg']},
             scoring='f1_weighted', verbose=3)

In [11]:
y_pred=grid.predict(x_test) #(.predict->defined function)
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test,y_pred)
print(cm)

[[75  4]
 [10 31]]


In [12]:
from sklearn.metrics import classification_report
clf_report=classification_report(y_test,y_pred)
print(clf_report)

              precision    recall  f1-score   support

           0       0.88      0.95      0.91        79
           1       0.89      0.76      0.82        41

    accuracy                           0.88       120
   macro avg       0.88      0.85      0.87       120
weighted avg       0.88      0.88      0.88       120



In [13]:
re=grid.cv_results_
grid_predictions = grid.predict(x_test) 
from sklearn.metrics import f1_score
f1_macro=f1_score(y_test,y_pred,average='weighted')
print("f1_macro value for {}:".format(grid.best_params_),f1_macro)
from sklearn.metrics import roc_auc_score
roc_auc_score(y_test,grid.predict_proba(x_test)[:,1])
table=pd.DataFrame.from_dict(re)
table

f1_macro value for {'penalty': 'l1', 'solver': 'liblinear'}: 0.8808622165169021


,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_penalty,param_solver,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.002839,0.004151,0.000000,0.000000,l1,lbfgs,"{'penalty': 'l1', 'solver': 'lbfgs'}",NaN,NaN,NaN,NaN,NaN,NaN,NaN,10
1,0.013886,0.004584,0.012362,0.001793,l1,liblinear,"{'penalty': 'l1', 'solver': 'liblinear'}",0.855314,0.802399,0.644599,0.927778,0.890114,0.824041,0.098765,1
2,0.006440,0.003720,0.014421,0.002292,l1,saga,"{'penalty': 'l1', 'solver': 'saga'}",0.835985,0.782971,0.644599,0.927778,0.890114,0.816289,0.098849,6
3,0.004521,0.004991,0.000000,0.000000,l1,newton-cg,"{'penalty': 'l1', 'solver': 'newton-cg'}",NaN,NaN,NaN,NaN,NaN,NaN,NaN,10
4,0.018962,0.005712,0.011162,0.003013,l2,lbfgs,"{'penalty': 'l2', 'solver': 'lbfgs'}",0.855314,0.782971,0.644599,0.927778,0.870721,0.816277,0.097471,7
5,0.009018,0.001772,0.009262,0.004000,l2,liblinear,"{'penalty': 'l2', 'solver': 'liblinear'}",0.855314,0.802399,0.644599,0.927778,0.870721,0.820162,0.096448,2
6,0.009056,0.004646,0.014166,0.004941,l2,saga,"{'penalty': 'l2', 'solver': 'saga'}",0.855314,0.782971,0.644599,0.927778,0.870721,0.816277,0.097471,7
7,0.015252,0.004524,0.011745,0.001610,l2,newton-cg,"{'penalty': 'l2', 'solver': 'newton-cg'}",0.855314,0.782971,0.644599,0.927778,0.870721,0.816277,0.097471,7
8,0.000000,0.000000,0.000000,0.000000,elasticnet,lbfgs,"{'penalty': 'elasticnet', 'solver': 'lbfgs'}",NaN,NaN,NaN,NaN,NaN,NaN,NaN,10
9,0.002251,0.004017,0.000000,0.000000,elasticnet,liblinear,"{'penalty': 'elasticnet', 'solver': 'liblinear'}",NaN,NaN,NaN,NaN,NaN,NaN,NaN,10
